# LLM-based Data Enrichment

## Imports

In [4]:
import pandas as pd
import os
from dotenv import load_dotenv
from openai import OpenAI
from google import genai
import json
import time
import re

## Pre-proccessing

To enable LLM-based enrichment, an OpenAI API key was configured and made available to the runtime environment via environment variables. For security reasons, the key is not embedded in the notebook.

In [5]:
load_dotenv()
assert os.getenv("OPENAI_API_KEY"), "OPENAI_API_KEY is missing. Check .env location/name."
key = os.getenv("OPENAI_API_KEY")
df = pd.read_csv("data/50_cps_models.csv")
df.head(10)

,model
0,APC 0N-9582PANTERA
1,APC AP7811B
2,APC AP9630
3,APC AP9631
4,APC ATS
5,APC MasterSwitchrPDU
6,APC MasterSwitchrPDU2
7,APC Smart-UPS 1500
8,APC Smart-UPS SMT 1500
9,APC Smart-UPS SRT 2200


We initialize the OpenAI client using environment-based configuration and define a closed vocabulary of device types to constrain the classification output:

In [6]:
gpt_client = OpenAI()

#create a controlled vocabulary of device types
allowed = {"PLC", "HMI", "Industrial PC (IPC)", "Building Management Controller (BMS)", "Power Distribution Unit (PDU)", "UPS", "Power Controller", "Medical Infusion Pump", "Medical Patient Monitor", "Laboratory Analyzer", "Nurse Call System", "Network Appliance", "SCADA Server", "Safety Controller", "Sensor", "Camera", "Other", "Unknown"}


## Device Type Enrichment Using GPT

The following helper functions implement the device type enrichment workflow:
extract_json_array defensively parses GPT responses to recover a valid JSON array and convert it into Python objects.
classify_device_types_batch submits a batch of device models to the GPT model, enforces deterministic inference, and validates the structured output.
classify_all_device_types applies the batch classifier across the full dataset, processes models in controlled batches, and returns the inferred device types, confidence levels, and justifications as aligned Series objects.

The prompt itself is defined as a structured string that constrains the model’s role, output schema, and allowed vocabulary to ensure consistent, inventory-compatible classifications.

In [7]:
device_type_enrichment_prompt = """
You are an OT security analyst responsible for enriching CPS asset inventories.

Task:
For each CPS device model listed below, identify the most appropriate device type as it would
appear in a real OT/ICS asset inventory.

Important context:
- Device type attribution is generally stable and vendor-intended.
- Many CPS product families have a dominant, well-established role in industry practice.
- This task reflects asset inventory enrichment, not exhaustive functional analysis.

Guidelines:
- Prefer a specific, domain-aware category (product-family semantics) rather than a generic label.
- If a model could plausibly fit multiple roles, choose the most common primary role in practice.
- Use industry-standard defaults when a product family is strongly associated with a device type.
- Do not overuse "Other" as a fallback when a reasonable industry classification exists.
- Use "Other" only when the model clearly does not align with any listed category.
- Use "Unknown" only when you cannot reasonably infer the device type at all.

Confidence calibration guidance:
- Use "high" confidence when the device type is strongly and consistently associated
  with the vendor and product family.
- Use "medium" confidence when relying on common industry defaults or partial signals.
- Use "low" confidence when the inference is weak or indirect.
- "Other" and "Unknown" should generally be paired with "low" confidence.

Allowed device types (use only these exact values):
PLC, HMI, Industrial PC (IPC), Building Management Controller (BMS),
Power Distribution Unit (PDU), UPS, Power Controller,
Medical Infusion Pump, Medical Patient Monitor, Laboratory Analyzer, Nurse Call System,
Network Appliance, SCADA Server, Safety Controller, Sensor, Camera, Other, Unknown

Output format:
Return a JSON array of objects, one per model, in the same order.
Each object must have:
- "device_type": one of the allowed device types
- "confidence": one of ["low", "medium", "high"]
- "justification": short reasoning based on product-family semantics (1 sentence)

Return ONLY valid JSON. No markdown. No extra text.

Models:
"""

In [8]:
# a function to extract the first JSON array from a string and parse it
def extract_json_array(text):
    if not text or not text.strip():
        raise ValueError("Empty LLM response")
    text = text.strip()
    # Defensive: handle markdown fences if they appear
    if text.startswith("```"):
        parts = text.split("```")
        if len(parts) >= 2:
            text = parts[1].strip()
        if text.lower().startswith("json"):
            text = text[4:].strip()
    start = text.find("[")
    end = text.rfind("]")
    if start == -1 or end == -1:
        raise ValueError(f"No JSON array found in response:\n{text}")
    return json.loads(text[start:end + 1])

In [9]:
# a function to classify device types for a batch of model names
def classify_device_types_batch(models_batch):
    prompt = device_type_enrichment_prompt + "\n".join(models_batch)
    response = gpt_client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0,
        max_tokens=900,
        messages=[
            {"role": "system", "content": "Return only valid JSON. No markdown."},
            {"role": "user", "content": prompt},
        ],
    )

    parsed = extract_json_array(response.choices[0].message.content)

    device_types = []
    confidences = []
    justifications = []

    for obj in parsed:
        dt = obj.get("device_type", "Unknown")
        conf = obj.get("confidence", "low")
        just = obj.get("justification", "")

        if dt not in allowed:
            dt = "Unknown"
        if conf not in {"low", "medium", "high"}:
            conf = "low"
        if not isinstance(just, str):
            just = str(just)

        device_types.append(dt)
        confidences.append(conf)
        justifications.append(just.strip())

    return device_types, confidences, justifications

In [10]:
# apply the device classification to the full dataset
def classify_all_device_types(df, batch_size=10):
    models = df["model"].astype(str).tolist()

    device_types_all = []
    confidences_all = []
    justifications_all = []

    total_batches = (len(models) + batch_size - 1) // batch_size

    for i in range(0, len(models), batch_size):
        batch = models[i:i + batch_size]
        print(f"Processing batch {i // batch_size + 1}/{total_batches}")

        dts, confs, justs = classify_device_types_batch(batch)

        device_types_all.extend(dts)
        confidences_all.extend(confs)
        justifications_all.extend(justs)

        time.sleep(0.2)

    return (
        pd.Series(device_types_all, index=df.index),
        pd.Series(confidences_all, index=df.index),
        pd.Series(justifications_all, index=df.index),
    )


#apply the enrichment
df["device_type"], df["device_type_confidence"], df["device_type_justification"] = classify_all_device_types(df)
df.head()

Processing batch 1/5
Processing batch 2/5
Processing batch 3/5
Processing batch 4/5
Processing batch 5/5


,model,device_type,device_type_confidence,device_type_justification
0,APC 0N-9582PANTERA,Power Distribution Unit (PDU),high,The APC 0N-9582PANTERA is a model associated w...
1,APC AP7811B,Power Distribution Unit (PDU),high,The APC AP7811B is a well-known model of a pow...
2,APC AP9630,Power Controller,medium,The APC AP9630 is commonly used as a network m...
3,APC AP9631,Power Controller,medium,The APC AP9631 serves as a network management ...
4,APC ATS,Power Distribution Unit (PDU),high,The APC ATS is recognized as an automatic tran...


In [11]:
#saving the dataframe
df=df.sort_values(by=["device_type","device_type_confidence"])
df.to_csv("output\Cps_Models_DeviceType_enrichment_byGPT.csv", index=False)
display(df.groupby("device_type").size().sort_values(ascending=False).rename("count").reset_index())
df.groupby("device_type_confidence").size().sort_values(ascending=False).rename("count").reset_index()

,device_type,count
0,Other,8
1,Medical Infusion Pump,8
2,Building Management Controller (BMS),7
3,Power Distribution Unit (PDU),6
4,Medical Patient Monitor,6
5,UPS,4
6,HMI,3
7,Industrial PC (IPC),2
8,Power Controller,2
9,Network Appliance,2


,device_type_confidence,count
0,high,33
1,medium,9
2,low,8


Repeated executions of the LLM-based device type enrichment resulted in minor variations in borderline categories (e.g., Other vs. specific controller types). However, high-level device type assignments remained stable across runs, and no devices shifted across different domains. This indicates sufficient reliability for asset inventory enrichment, where ambiguous cases are expected and explicitly captured via confidence levels.

The LLM-based device type enrichment demonstrates high overall coverage and reliability across the dataset. The vast majority of devices are mapped to concrete, domain-specific categories, with only a small residual portion falling into generic or ambiguous classifications. The resulting distribution is dominated by medically and operationally meaningful asset types, alongside core OT infrastructure components such as controllers, industrial computing platforms, and power-related devices. This aligns well with the expected composition of a mixed CPS environment and suggests that the model successfully captures vendor and product-family semantics.

From a confidence perspective, most classifications are assigned high confidence, indicating that device model naming conventions and industry norms provide strong signals for deterministic inference. A smaller subset is labeled with medium confidence, typically reflecting reliance on common industry defaults or partial semantic cues, while low-confidence cases remain limited and are explicitly surfaced. Variability across repeated executions is constrained to closely related categories (for example, generic “Other” versus a specific controller type, or IPC versus Network Appliance) and is concentrated primarily within medium- and low-confidence predictions.

Crucially, no classifications were observed to shift across high-level semantic domains, such as between medical devices, industrial control systems, and IT/network infrastructure. This indicates strong semantic stability of the enrichment process even under minor prompt or execution variability. Overall, the results suggest that the LLM provides robust, interpretable device type enrichment suitable for asset inventory and risk prioritization workflows, with confidence annotations enabling targeted analyst review where ambiguity remains.

## OS Enrichment Using GPT

First, let's build the main function for this enrichment which will include the prompt, the response object and the json extraction:

In [12]:
allowed_OS_families = {"Linux","Windows Embedded","VxWorks","Unknown"}
os_enrichment_prompt = """
        You are an OT security analyst enriching CPS asset inventories with operating system information.
        
        Task:
        For each CPS device listed below, infer the most likely operating system *family*
        based on common vendor practices and typical deployments.
        
        Important context:
        - OS attribution is probabilistic and less stable than device type.
        - Device type information is already known and should be used to inform OS inference.
        - This task reflects asset intelligence enrichment, not definitive fingerprinting.
        
        Instructions:
        - Infer an OS family only when it is commonly associated with the product family
          and the provided device type.
        - Do not assume custom, lab-specific, or rare configurations.
        - If multiple OS families are plausible, choose the most common one and reduce confidence.
        - If the OS cannot be reasonably inferred, use "Unknown".
        
        Allowed OS families (use only these exact values):
        Linux, Windows Embedded, VxWorks, Unknown
        
        Output format:
        Return a JSON array of objects, one per entry, in the same order as provided.
        Each object must include:
        - "os_family": one of the allowed OS families
        - "confidence": one of ["low", "medium", "high"]
        - "justification": short factual reasoning based on vendor, product family,
          and the provided device type
        
        Return ONLY valid JSON. No markdown. No extra text.
        
        Devices (model | known device type):
        """ 

In [13]:
def classify_os_batch(models_with_device_type):
    prompt = os_enrichment_prompt + "\n".join(models_with_device_type)
    response = gpt_client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0,
        max_tokens=900,
        messages=[
            {"role": "system", "content": "Return only valid JSON. No markdown."},
            {"role": "user", "content": prompt},
        ],
    )

    parsed = extract_json_array(response.choices[0].message.content)

    os_vals, conf_vals, just_vals = [], [], []

    for obj in parsed:
        os_family = obj.get("os_family", "Unknown")
        confidence = obj.get("confidence", "low")
        justification = obj.get("justification", "")

        if os_family not in allowed_OS_families:
            os_family = "Unknown"
        if confidence not in {"low", "medium", "high"}:
            confidence = "low"
        if not isinstance(justification, str):
            justification = str(justification)

        os_vals.append(os_family)
        conf_vals.append(confidence)
        just_vals.append(justification.strip())

    return os_vals, conf_vals, just_vals

Now, we need a function that uses the device type from the previous enrichment and a function for the batching:

In [14]:
def build_model_context(df, start, end):
    return [
        f"{row['model']} | device_type: {row['device_type']}"
        for _, row in df.iloc[start:end].iterrows()
    ]


# apply OS inference to full dataset
def enrich_os_on_existing_df_gpt(df, batch_size=10):
    os_all, conf_all, just_all = [], [], []

    total_batches = (len(df) + batch_size - 1) // batch_size

    for i in range(0, len(df), batch_size):
        print(f"Processing batch {i // batch_size + 1}/{total_batches}")

        models_with_device_type = build_model_context(df, i, i + batch_size)

        os_vals, conf_vals, just_vals = classify_os_batch(models_with_device_type)

        os_all.extend(os_vals)
        conf_all.extend(conf_vals)
        just_all.extend(just_vals)

        time.sleep(0.2)

    df["os_family_byGPT"] = os_all
    df["os_confidence_byGPT"] = conf_all
    df["os_justification_byGPT"] = just_all

    return df

# run OS enrichment
df = enrich_os_on_existing_df_gpt(df)
display(df.head())

Processing batch 1/5
Processing batch 2/5
Processing batch 3/5
Processing batch 4/5
Processing batch 5/5


,model,device_type,device_type_confidence,device_type_justification,os_family_byGPT,os_confidence_byGPT,os_justification_byGPT
21,Carel PCO1000WB0,Building Management Controller (BMS),high,The Carel PCO1000WB0 is specifically designed ...,Unknown,low,Carel devices typically use proprietary softwa...
25,Hill-Rom NaviCare Room Control Board,Building Management Controller (BMS),high,The Hill-Rom NaviCare Room Control Board is us...,Unknown,low,Hill-Rom devices often run on proprietary syst...
26,Honeywell ComfortPoint Open CPO-VAV2A,Building Management Controller (BMS),high,The Honeywell ComfortPoint Open CPO-VAV2A is d...,Unknown,low,Honeywell ComfortPoint devices are known for p...
29,Johnson Controls NAE,Building Management Controller (BMS),medium,The Johnson Controls NAE is used for building ...,Unknown,low,Johnson Controls devices often utilize proprie...
43,Schneider Electric AS series,Building Management Controller (BMS),medium,The Schneider Electric AS series is commonly u...,Unknown,low,Schneider Electric AS series devices typically...


In [15]:
df = df.sort_values(by=["os_family_byGPT","os_confidence_byGPT"])
display(df.head())
display(df.groupby("os_family_byGPT").size().sort_values(ascending=False).rename("count").reset_index())
df.groupby("os_confidence_byGPT").size().sort_values(ascending=False).rename("count").reset_index()

,model,device_type,device_type_confidence,device_type_justification,os_family_byGPT,os_confidence_byGPT,os_justification_byGPT
28,IGEL IGEL Thin Client,Industrial PC (IPC),medium,The IGEL IGEL Thin Client is often used in ind...,Linux,medium,IGEL Thin Clients often use Linux-based operat...
30,Lantronix MatchPort AR,Network Appliance,high,The Lantronix MatchPort AR is primarily used f...,Linux,medium,Lantronix MatchPort AR is typically associated...
22,Carel pCOWeb Card,Network Appliance,medium,The Carel pCOWeb Card is used for network conn...,Linux,medium,Carel pCOWeb Card is commonly found with Linux...
2,APC AP9630,Power Controller,medium,The APC AP9630 is commonly used as a network m...,Linux,medium,The APC AP9630 is commonly associated with Lin...
3,APC AP9631,Power Controller,medium,The APC AP9631 serves as a network management ...,Linux,medium,The APC AP9631 is typically deployed with Linu...


,os_family_byGPT,count
0,Unknown,35
1,Windows Embedded,10
2,Linux,5


,os_confidence_byGPT,count
0,low,35
1,medium,14
2,high,1


Most of the classificaitons are "Unknown", so we can try to improve the prompt in order to get better results:

In [16]:
#improvement of the previous os_enrichment_prompt
os_enrichment_prompt = """
        You are an OT security analyst enriching CPS asset inventories with operating system information.
        
        Task:
        For each CPS device listed below, infer the most likely operating system *family*
        based on common vendor practices and typical deployments.
        
        Important context:
        - OS attribution is probabilistic and less stable than device type.
        - Device type information is already known and should be used to inform OS inference.
        - This task reflects asset intelligence enrichment, not definitive fingerprinting.

        Additional guidance (industry OS priors):

        When no model-specific contradiction exists, you may rely on common industry defaults:
        - HMIs are commonly Windows Embedded.
        - Industrial PCs (IPC) commonly run Linux or Windows Embedded.
        - SCADA servers commonly run Windows Embedded or Linux.
        - Network appliances commonly run embedded Linux.
        - PLCs and safety controllers commonly run VxWorks or proprietary RTOS.
        - Building Management Controllers often run embedded Linux or proprietary firmware.
        
        If using such defaults, reflect this by setting confidence to "medium",
        unless the product line strongly indicates otherwise.

        Confidence calibration guidance:

        - Use "high" confidence when the OS family is strongly and consistently associated
          with the vendor and device type (e.g., mainstream HMIs, well-known PLC families).
        - Use "medium" confidence when relying on common industry defaults or device-type priors.
        - Use "low" confidence when the inference is weak, indirect, or based on limited signals.
        - "Unknown" should generally be paired with "low" confidence.

        Instructions:
        - Infer an OS family only when it is commonly associated with the product family
          and the provided device type.
        - Do not assume custom, lab-specific, or rare configurations.
        - If multiple OS families are plausible, choose the most common one and reduce confidence.
        - If the OS cannot be reasonably inferred, use "Unknown".
        - If the device type is known and a common OS family is strongly implied by industry practice,
            prefer assigning that OS family with "medium" confidence rather than defaulting to "Unknown".

        Allowed OS families (use only these exact values):
        Linux, Windows Embedded, VxWorks, Unknown
        
        Output format:
        Return a JSON array of objects, one per entry, in the same order as provided.
        Each object must include:
        - "os_family": one of the allowed OS families
        - "confidence": one of ["low", "medium", "high"]
        - "justification": short factual reasoning based on vendor, product family,
          and the provided device type 
        
        Return ONLY valid JSON. No markdown. No extra text.
        
        Devices (model | known device type):
        """ 

df = enrich_os_on_existing_df_gpt(df)

Processing batch 1/5
Processing batch 2/5
Processing batch 3/5
Processing batch 4/5
Processing batch 5/5


In [17]:
display(df.groupby("os_family_byGPT").size().sort_values(ascending=False).rename("count").reset_index())
display(df.groupby("os_confidence_byGPT").size().sort_values(ascending=False).rename("count").reset_index())
df.to_csv("output\Cps_Models_DeviceType+OsEnrichment_byGPT.csv",index=False)

,os_family_byGPT,count
0,Unknown,38
1,Linux,9
2,Windows Embedded,3


,os_confidence_byGPT,count
0,low,36
1,medium,9
2,high,5


We observe that the revised prompt improved the enrichment results, approximately doubling the number of classifications for one of the OS families. Nevertheless, “Unknown” remains the most frequent category, indicating that a substantial portion of devices still lack sufficient information for confident OS inference.

As a post-enrichment validation step, we should enforce basic consistency rules between device type and inferred OS family in order to have a sanity check. Classifications that violate well-known operational constraints will be conservatively downgraded to Unknown to preserve result reliability:
- Industrial PCs: Windows or Linux only
- PLCs: never Windows
- HMIs: rarely VxWorks
- Medical patient monitors: rarely full Linux

In [18]:
def validate_os_consistency(device_type, os_family):
    if device_type == "Industrial PC (IPC)":
        if os_family not in {"Linux", "Windows Embedded"}:
            return "Unknown"
    if device_type == "PLC":
        if os_family == "Windows Embedded":
            return "Unknown"
    if device_type == "HMI":
        if os_family == "VxWorks":
            return "Unknown"
    if device_type == "Medical Patient Monitor":
        if os_family == "Linux":
            return "Unknown"
    return os_family

In [19]:
#create a series of validated values of the OS labels
validated_os = df.apply(
    lambda r: validate_os_consistency(
        r["device_type"],
        r["os_family_byGPT"]
    ),
    axis=1
)

#creata series of boolean values
changed_mask = df["os_family_byGPT"] != validated_os
display(changed_mask.value_counts())
#change the values in the df only for rows where changed_mask == True
df.loc[changed_mask, "os_family_byGPT"] = validated_os[changed_mask]
display(df.groupby("os_family_byGPT").size().sort_values(ascending=False).rename("count").reset_index())

False    50
Name: count, dtype: int64

,os_family_byGPT,count
0,Unknown,38
1,Linux,9
2,Windows Embedded,3


The post-enrichment validation step did not alter any GPT-inferred OS labels. All classifications were consistent with predefined OT device-type constraints.

## OS Enrichment Using Gemini

Now, we perform OS enrichment using the same refined prompt, this time with the Gemini model.
To ensure a fair and controlled comparison, the Gemini API key is loaded via the environment in the same manner as the OpenAI key.

Although newer Gemini Flash variants are available, we intentionally select Gemini-1.5-Pro, prioritizing conservative reasoning, structured JSON output, and result stability over latency. These characteristics are better aligned with CPS asset enrichment and inventory intelligence tasks.


In [20]:
load_dotenv()
assert os.getenv("GEMINI_API_KEY"), "GEMINI_API_KEY is missing. Check .env location/name."
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
gemini_client = genai.Client(api_key=GEMINI_API_KEY)
GEMINI_MODEL = "gemini-2.5-flash"

In [21]:
def classify_os_batch_gemini(models_with_device_type):
    prompt = os_enrichment_prompt + "\n".join(models_with_device_type)

    response = gemini_client.models.generate_content(
    model=GEMINI_MODEL,
    contents=prompt
    )


    parsed = extract_json_array(response.text)

    os_vals, conf_vals, just_vals = [], [], []

    for obj in parsed:
        os_family = obj.get("os_family", "Unknown")
        confidence = obj.get("confidence", "low")
        justification = obj.get("justification", "")

        if os_family not in allowed_OS_families:
            os_family = "Unknown"
        if confidence not in {"low", "medium", "high"}:
            confidence = "low"
        if not isinstance(justification, str):
            justification = str(justification)

        os_vals.append(os_family)
        conf_vals.append(confidence)
        just_vals.append(justification.strip())

    return os_vals, conf_vals, just_vals

In [22]:
def enrich_os_on_existing_df_gemini(df, batch_size=10, max_retries=5):
    df = df.copy()

    df["os_family_byGemini"] = None
    df["os_confidence_byGemini"] = None
    df["os_justification_byGemini"] = None

    total_batches = (len(df) + batch_size - 1) // batch_size

    for i in range(0, len(df), batch_size):
        batch_idx = i // batch_size + 1
        print(f"Processing batch {batch_idx}/{total_batches}")

        batch_df = df.iloc[i:i + batch_size]

        # Skip already-enriched rows (resume-safe)
        if batch_df["os_family_byGemini"].notna().all():
            print("Batch already enriched, skipping")
            continue

        models_with_device_type = build_model_context(df, i, i + batch_size)

        for attempt in range(max_retries):
            try:
                os_vals, conf_vals, just_vals = classify_os_batch_gemini(models_with_device_type)
                break
            except Exception as e:
                if attempt == max_retries - 1:
                    print("Final failure on batch, stopping early")
                    return df
                wait = 2 ** attempt
                print(f"Gemini overloaded, retrying in {wait}s...")
                time.sleep(wait)

        df.loc[batch_df.index, "os_family_byGemini"] = os_vals
        df.loc[batch_df.index, "os_confidence_byGemini"] = conf_vals
        df.loc[batch_df.index, "os_justification_byGemini"] = just_vals

        time.sleep(1.0)

    return df

df= enrich_os_on_existing_df_gemini(df)
df.head()

Processing batch 1/5
Processing batch 2/5
Processing batch 3/5
Processing batch 4/5
Processing batch 5/5


,model,device_type,device_type_confidence,device_type_justification,os_family_byGPT,os_confidence_byGPT,os_justification_byGPT,os_family_byGemini,os_confidence_byGemini,os_justification_byGemini
28,IGEL IGEL Thin Client,Industrial PC (IPC),medium,The IGEL IGEL Thin Client is often used in ind...,Linux,medium,Industrial PCs (IPC) commonly run Linux or Win...,Linux,high,"IGEL is a thin client vendor, and their primar..."
30,Lantronix MatchPort AR,Network Appliance,high,The Lantronix MatchPort AR is primarily used f...,Linux,high,Network appliances typically run embedded Linu...,Linux,medium,Lantronix MatchPort AR is an embedded device s...
22,Carel pCOWeb Card,Network Appliance,medium,The Carel pCOWeb Card is used for network conn...,Linux,high,Network appliances typically run embedded Linu...,Linux,medium,The pCOWeb Card is an embedded network communi...
2,APC AP9630,Power Controller,medium,The APC AP9630 is commonly used as a network m...,Unknown,low,Power controllers like APC AP9630 do not have ...,Linux,medium,The APC AP9630 is an embedded network manageme...
3,APC AP9631,Power Controller,medium,The APC AP9631 serves as a network management ...,Unknown,low,Power controllers like APC AP9631 do not have ...,Linux,medium,The APC AP9631 is an embedded network manageme...


In [23]:
df = df.sort_values(by=["os_family_byGemini","os_confidence_byGemini"])
df.to_csv("output\Cps_Models_DeviceType+OsEnrichment_byGemini.csv",index=False)
display(df.groupby("os_family_byGemini").size().sort_values(ascending=False).rename("count").reset_index())
df.groupby("os_confidence_byGemini").size().sort_values(ascending=False).rename("count").reset_index()

,os_family_byGemini,count
0,Linux,28
1,Windows Embedded,11
2,VxWorks,8
3,Unknown,3


,os_confidence_byGemini,count
0,medium,40
1,high,5
2,low,5


In [24]:
#create a series of validated values of the OS labels
validated_os = df.apply(
    lambda r: validate_os_consistency(
        r["device_type"],
        r["os_family_byGemini"]
    ),
    axis=1
)

#creata series of boolean values
changed_mask = df["os_family_byGemini"] != validated_os
display(changed_mask.value_counts())
#change the values in the df only for rows where changed_mask == True
df.loc[changed_mask, "os_family_byGemini"] = validated_os[changed_mask]
display(df.groupby("os_family_byGemini").size().sort_values(ascending=False).rename("count").reset_index())

False    46
True      4
Name: count, dtype: int64

,os_family_byGemini,count
0,Linux,24
1,Windows Embedded,11
2,VxWorks,8
3,Unknown,7


Again, the validation step did not alter any GPT-inferred OS labels.

Now, let's evaluate the two models by KPIs:

In [25]:
def coverage_rate(series):
    return (series != "Unknown").mean()
    
def usable_confidence_rate(conf_series):
    return conf_series.isin({"medium", "high"}).mean()
    
def agreement_rate(a, b):
    return (a == b).mean()

coverage_gpt = coverage_rate(df["os_family_byGPT"])
coverage_gemini = coverage_rate(df["os_family_byGemini"])

usable_gpt = usable_confidence_rate(df["os_confidence_byGPT"])
usable_gemini = usable_confidence_rate(df["os_confidence_byGemini"])

agreement = agreement_rate(
    df["os_family_byGPT"],
    df["os_family_byGemini"]
)

In [26]:
model_metrics = pd.DataFrame({
    "Metric": ["Coverage rate", "Usable confidence rate (medium+high)"],
    "GPT": [coverage_gpt, usable_gpt],
    "Gemini": [coverage_gemini, usable_gemini]
})
display(model_metrics.style.format({
        "GPT": "{:.2%}",
        "Gemini": "{:.2%}"}))
agreement_table = pd.DataFrame({"Metric": ["Model agreement rate"], "Value": [agreement]})
display(agreement_table.style.format({
        "Value": "{:.2%}"}))
output_path = "output/models_comparison.xlsx"
with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
    model_metrics.to_excel(writer, sheet_name="Model metrics", index=False)
    agreement_table.to_excel(writer, sheet_name="Agreement", index=False)


,Metric,GPT,Gemini
0,Coverage rate,24.00%,86.00%
1,Usable confidence rate (medium+high),28.00%,90.00%


,Metric,Value
0,Model agreement rate,36.00%


Compared to the GPT-based enrichment, the Gemini-based enrichment yields a more balanced and informative OS family distribution. The dominance of Unknown is noticeably reduced, while multiple OT-relevant OS families are identified more consistently. This indicates that the Gemini prompt and model better exploit vendor-and domain-specific signals instead of defaulting to uncertainty, resulting in broader categorical coverage.

**One model clearly prioritizes coverage, assigning OS labels to a much larger portion of the inventory, while the other is more conservative, leaving more assets as Unknown. This indicates that the difference is philosophical, not technical: one model is optimized to infer aggressively from weak signals, the other to avoid over-commitment when evidence is limited**.

The confidence distribution further supports this improvement: Gemini outputs are concentrated around medium-to-high confidence levels, whereas GPT tends to produce a larger share of low-confidence classifications. This suggests that Gemini’s predictions are not only more diverse but also more decisive and internally consistent. Overall, the Gemini-based enrichment appears better suited for practical CPS/OT inventory enrichment, even though a residual Unknown category remains unavoidable in the absence of ground truth.

**The large gap in usable confidence (medium+high) shows that one model is willing to stand behind its predictions more often. However, higher confidence does not imply higher correctness in the absence of ground truth. This metric therefore measures assertiveness, not accuracy — which is critical for security inventories where false certainty is costly**.

The observed agreement rate indicates that a substantial subset of CPS assets has inherently ambiguous OS characteristics when inferred from model metadata alone. Disagreements between models highlight cases where conservative handling or further validation is required, rather than a failure of either model.